Lib config

In [ ]:
!pip install gspread oauth2client
!pip install slackclient

In [ ]:
import slack
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import itertools
import re
import json
from pathlib import Path
import string

Sheets Setup

In [ ]:
# run this cell twice so that empty sender cells get filled

scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name("key.json", scope)

client = gspread.authorize(creds)

# sheet = client.open_by_key("1R7m9iV5D1Z1suMWybkbmumwxWrNVZbikFIGYAw5SqO4").sheet1 # <- Mock TKSugar
sheet = client.open_by_key("1h4PzQIynyQixCEp0tE0PJXOrXtDxGXriPBi-cZYm9ck").sheet1  # <- TKSugar (Responses)

#collect all values from given columns
timestamp = sheet.col_values(1)
recipient = sheet.col_values(2)
message = sheet.col_values(3)
identity = sheet.col_values(4)
sendstatus = sheet.col_values(5)

#loop through empty sender cells and write filler text
for x in range(len(message)):
  y = sheet.cell(col=4, row=x+1)
  if y.value == None or not y.value:
    sheet.update_cell(col=4, row= x+1, value= "A Secret Someone...")

Sheets Scrape

In [ ]:
#translator for unicode
escapes = ''.join([chr(char) for char in range(1, 32)])
translator = str.maketrans('', '', escapes)

#create 2d array with each messages info
peeps = []
for (x, y, z, t, s, n) in itertools.zip_longest(recipient, message, identity, timestamp, sendstatus, range(len(message))):
    x = ((x.lower()).strip()).replace(',', '')
    y = y.translate(translator)
    xy = []
    xy.extend([x, str(y), z, t, n+1, s])
    peeps.append(xy)

#pop 1st index to del header row from array
peeps.pop(0)

#popping all indexes with sent status
for person in list(peeps):

  if person[5] == "sent":
    peeps.pop(peeps.index(person))

print(peeps)


Debug sheets


In [ ]:
escapes = ''.join([chr(char) for char in range(1, 32)])
translator = str.maketrans('', '', escapes)
# .translate(translator)

Slack Main


In [ ]:
SLACK_TOKEN= "XXXXX-427480842549-1597618771813-Dsq5xvSnD1PTzTBXk8Tfy6w0"

client = slack.WebClient(SLACK_TOKEN)

users_store = {}
users_first = {}

result = client.users_list(workspace= "TCKE4QSG5")

#array with all member info
save_users=(result["members"])

#loop through member info to create 2 lists with member ids
for user in save_users:
  user_name = user["profile"]["real_name"]
  x = user_name.split()

#if member info includes both first and last name, append to this list
  if len(x) > 1:
    users_store[user_name.lower()] = user["id"]

#if member info only includes first name, append to this list
  else:
     users_first[user_name.lower()] = user["id"]

#looping through 2d array
for person in peeps:

  name = person[0]
  message = person[1]
  sender = person[2]

#first tries to search for member id in array with both first name and last name as key
  try:
    person_ID = [val for key, val in users_store.items() if name in key]

    client.chat_postMessage(channel= person_ID[0], text =("""You recieved an amazing valentines wish from {}
    {}""").format(sender, message))

#updates sent status in spreadsheet
    sheet.update_cell(col=5, row= int(person[4]), value= "sent")

    print(person_ID, name, " Success!")

  except IndexError:

#if first search fails, tries to seach for member id in array with only first name as key
    try:
      x = name.split()
      name = x[0]
      person_ID = [val for key, val in users_first.items() if name in key]

      client.chat_postMessage(channel= person_ID[0], text =("""You recieved an amazing valentines wish from {}
      {}""").format(sender, message))

#updates sent status in spreadsheet
      sheet.update_cell(col=5, row= int(person[4]), value= "sent")

      print(person_ID, name, " Success!")

    except IndexError:
      print("Failed to send! Either name is spelt wrong or {} is outside of TKS Toronto".format(name))



Debug Slack



In [ ]:
name = peeps[0][0]
message = peeps[0][1]
sender = peeps[0][2]
person_ID = "U01A54SSBNE"

client.chat_postMessage(channel= "U01A54SSBNE", text =("""You received an amazing valentines wish from {}
{}""").format(sender, message))

sheet.update_cell(col=5, row= int(peeps[0][4]), value= "sent")
